In [1]:
import pandas as pd
import numpy as np
project_performance = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Project Performance')
kok_penalty = pd.read_excel('Tools KPI (v1).xlsx', sheet_name="Know or dont know")
absen_penalty = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Absensi')

Project Performance

In [2]:
def calculate_poin(x):
    if x['Project'] == 'Bravo Collection':
        return 4 if x['Komponen Project Performance 1'] > 0.7 else (3 if x['Komponen Project Performance 1'] >= 0.5 else (2 if x['Komponen Project Performance 1'] >= 0.3 else 1))
    elif x['Project'] == 'Bravo Routine Syariah':
        poin1 = 4 if x['Komponen Project Performance 1'] >= 1 else (3 if x['Komponen Project Performance 1'] >= 0.9 else (2 if x['Komponen Project Performance 1'] >= 0.75 else 1))
        poin2 = x['Komponen Project Performance 2']
        poin_syariah = (poin1 + poin2) / 2
        return poin_syariah
    elif x['Project'] == 'Bravo RO':
        poin1 = 4 if x['Komponen Project Performance 1'] >= 1 else (3 if x['Komponen Project Performance 1'] >= 0.8 else (2 if x['Komponen Project Performance 1'] >= 0.5 else 1))
        poin2 = 4 if x['Komponen Project Performance 2'] >= 0.95 else (3 if x['Komponen Project Performance 2'] >= 0.8 else (2 if x['Komponen Project Performance 2'] >= 0.5 else 1))
        poin3 = 4 if x['Komponen Project Performance 3'] >= 0.9 else (3 if x['Komponen Project Performance 3'] >= 0.7 else (2 if x['Komponen Project Performance 3'] >= 0.5 else 1))
        poin_RO = (poin1 + poin2 + poin3) / 3
        return poin_RO
    elif x['Project'] == 'After Care':
        poin_AC = x['Komponen Project Performance 1']
        return poin_AC
    elif x['Project'] == 'Idle':
        poin1 = 4 if x['Komponen Project Performance 1'] >= 90 else (3 if x['Komponen Project Performance 1'] >= 70 else (2 if x['Komponen Project Performance 1'] >= 50 else 1))
        poin_idle = poin1
        return poin_idle
    else:
        return None
project_performance['poin pp'] = project_performance.apply(calculate_poin, axis=1)


PENALTY

In [3]:
penalty = pd.merge(kok_penalty, absen_penalty, on=['NIK','NAMA TF'], how='outer')
penalty ['poin kok'] = penalty['Final Score'].apply(lambda y: 0.1 if y< 50 else (0.05 if y< 70 else 0))
penalty ['poin absen'] = penalty ['Total Keterlambatan'].apply(lambda y: 0 if y<3 else (0.15 if y<4 else (0.2 if y<=5 else 0.3)))
penalty ['bobot penalty'] = penalty ['poin kok'] + penalty ['poin absen']

Review and Feedback

In [4]:
review_user = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Review Feedback User')
review_spv = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Review Feedback Project')
review_idle = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Review Feedback non Project')
review_user = review_user.groupby('NAMA TF').mean()
rf_project =  pd.merge(review_user, review_spv, on=['NIK','NAMA TF'], how='outer')

C:\Users\082020\AppData\Local\Temp\ipykernel_2636\4269085039.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  review_user = review_user.groupby('NAMA TF').mean()


In [5]:
rf_project ['rf_poin'] = 0.4*rf_project['AVERAGE_USER'] + 0.6*rf_project['AVERAGE_SPV']
rf_project = rf_project [['NIK','NAMA TF','AVERAGE_USER','AVERAGE_SPV', 'rf_poin']]
review_idle ['rf_poin'] = review_idle ['AVERAGE']
rf_poin_all = pd.merge(rf_project, review_idle,on=['NIK','NAMA TF','rf_poin'], how='outer')
rf_poin_all = rf_poin_all [['NIK','NAMA TF','rf_poin']]

In [6]:
dakar = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='dakar')
dakar.rename(columns={'Name':'NAMA TF'}, inplace=True)
mapping = pd.read_excel('Tools KPI (v1).xlsx', sheet_name='Mappingan')

In [7]:
raport = project_performance[['NAMA TF','NIK']]
raport = raport.merge(project_performance[['NAMA TF','Final Compliance Report','poin pp']], on='NAMA TF', how='left')
raport = raport.merge(rf_poin_all[['NAMA TF','rf_poin']], on='NAMA TF', how= 'left')
raport = raport.merge(absen_penalty[['NAMA TF','Total Keterlambatan']] , on='NAMA TF', how='left')
raport = raport.merge(penalty[['NAMA TF','bobot penalty']], on='NAMA TF', how='left')
raport = raport.merge(dakar[['NAMA TF','JobTitle','Join Date','EmploymentStatus']], on='NAMA TF', how='left')
raport = raport.merge(mapping[['NAMA TF','JOIN AS TF','PERIODE','Project','Cabang']], on='NAMA TF', how='left')
raport['KPI FINAL'] = ((0.4 * (raport['rf_poin'] + raport['poin pp']) + 0.2 * (raport['Final Compliance Report'])) * (1 - raport['bobot penalty']))
raport = raport.drop(columns='bobot penalty')
raport['Sanksi History'] = None
raport['Sanksi Periode'] = None
raport['user_id'] = None
raport = raport[['NAMA TF','NIK','Join Date','JOIN AS TF','JobTitle','EmploymentStatus','Project','Cabang','PERIODE','poin pp','Final Compliance Report','rf_poin','Total Keterlambatan','KPI FINAL','Sanksi History','Sanksi Periode','user_id']]
raport.rename(columns={'NAMA TF':'NAME','Join Date':'JOIN DATE','JobTitle':'POSITION','EmploymentStatus':'EMPLOYEE STATUS','Project':'PROJECT','Cabang':'CABANG/AREA','poin pp':'BASELINE VS ACTUALIZATION','Final Compliance Report':'COMPLIANCE REPORT','rf_poin':'REVIEW DAN PENILAIAN SPV','Total Keterlambatan':'FATAL SCORE Discipline'},inplace=True)
raport

,NAME,NIK,JOIN DATE,JOIN AS TF,POSITION,EMPLOYEE STATUS,PROJECT,CABANG/AREA,PERIODE,BASELINE VS ACTUALIZATION,COMPLIANCE REPORT,REVIEW DAN PENILAIAN SPV,FATAL SCORE Discipline,KPI FINAL,Sanksi History,Sanksi Periode,user_id
0,Dwi Jenika Panggabean,93950,2022-02-16,2020-10-01,Project Implementation Specialist,Permanent,Bravo Routine Syariah,Bandung,2023-09,1.000000,2.856667,3.300000,1,2.291333,None,None,None
1,Ecky Pradana,97640,2023-01-09,2021-10-11,Project Implementation Specialist,Contract,Bravo Routine Syariah,"BINJAI, KARO, MEDAN, MEDAN 2, PEMATANG SIANTAR",2023-09,2.500000,2.793333,3.428571,3,2.490581,None,None,None
2,Debora Christa Romauli Simangunsong,92414,2021-10-11,2021-07-05,Project Implementation Specialist,Permanent,Bravo Routine Syariah,"MAMUJU UTARA, PALU",2023-09,3.000000,3.173333,3.471429,2,3.223238,None,None,None
3,Fransiska Malita Widyaningrum,97832,2023-01-09,2021-07-05,Project Implementation Specialist,Contract,Bravo Routine Syariah,"CIREBON, INDRAMAYU, KUNINGAN",2023-09,3.000000,1.656667,3.785714,1,3.045619,None,None,None
4,Amelia Maura,89120,2020-10-01,2019-12-03,Project Implementation Specialist,Permanent,Bravo Routine Syariah,"BALIKPAPAN, PASER, PENAJAM PASER UTARA",2023-09,3.000000,1.866667,3.057143,2,2.796190,None,None,None
5,Galiano Diego Armando,93958,2022-02-16,2021-10-11,Project Implementation Specialist,Permanent,Bravo Collection,"CIKAMPEK, KARAWANG",2023-09,3.000000,3.621250,3.664286,3,2.881470,None,None,None
6,Miftah Hidayat,96447,2022-09-21,2023-03-09,Project Implementation Specialist,Contract,Bravo Collection,"PALEMBANG, PALEMBANG 2, PANGKALAN BALAI",2023-09,1.000000,2.777273,3.450000,2,2.335455,None,None,None
7,Sabikhisma Noorfajr,93799,2022-02-16,1996-06-24,Project Implementation Specialist,Permanent,Bravo Collection,"Jaksel, jaksel 2, Jaksel 3",2023-09,2.000000,2.675000,2.750000,3,2.069750,None,None,None
8,Raymond Purba,97828,2023-01-09,2022-02-16,Project Implementation Specialist,Contract,Bravo Collection,"LUWUK, PARIGI - MOUTONG, POSO",2023-09,2.000000,1.538182,3.800000,2,2.627636,None,None,None
9,Yudhika Lewi Simorangkir,97833,2023-01-09,2023-03-15,Project Implementation Specialist,Contract,Bravo Collection,"BERAU, BULUNGAN, TARAKAN",2023-09,1.000000,1.771739,3.114286,1,2.000062,None,None,None


In [8]:
raport.to_excel('Raport.xlsx')